In [2]:
import json
import numpy as np
from model_balancing.model_balancing_cvx import ModelBalancingConvex, Q_
import cvxpy as cp
import pandas as pd
from sbtab import SBtab

In [4]:
with open("../cvxpy/examples/json/branch_point_model_artificial_noisefree_state_no_kinetic.json", "rt") as fp:
    data = json.load(fp)

keq_standard_concentration = Q_(data["standard_concentration"])

args = {}
args["S"] = np.array(data["network"]["stoichiometric_matrix"])
args["A_act"] = np.array(data["network"]["activation_matrix"])
args["A_inh"] = np.array(data["network"]["inhibition_matrix"])
args["fluxes"] = Q_(data["reaction_fluxes"]["data"]["mean"], data["reaction_fluxes"]["unit"])

args["Keq_gmean"] = Q_(data["kinetic_constants"]["Keq"]["combined"]["geom_mean"], "")

# "fix" the standard concentration (to a convention where it is 1 M which is used internally)
args["Keq_gmean"] *= np.power(keq_standard_concentration.m_as("M"), args["S"].sum(axis=0))

args["Keq_ln_cov"] = np.array(data["kinetic_constants"]["Keq"]["combined"]["cov_ln"])

args["kcatf_gmean"] = Q_(data["kinetic_constants"]["Kcatf"]["combined"]["geom_mean"], data["kinetic_constants"]["Kcatf"]["unit"])
args["kcatf_ln_cov"] = np.array(data["kinetic_constants"]["Kcatf"]["combined"]["cov_ln"])

args["kcatr_gmean"] = Q_(data["kinetic_constants"]["Kcatr"]["combined"]["geom_mean"], data["kinetic_constants"]["Kcatr"]["unit"])
args["kcatr_ln_cov"] = np.array(data["kinetic_constants"]["Kcatr"]["combined"]["cov_ln"])

args["Km_gmean"] = Q_(data["kinetic_constants"]["KM"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KM"]["unit"])
args["Km_ln_cov"] = np.array(data["kinetic_constants"]["KM"]["combined"]["cov_ln"])

args["Ka_gmean"] = Q_(data["kinetic_constants"]["KA"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KA"]["unit"])
args["Ka_ln_cov"] = np.array(data["kinetic_constants"]["KA"]["combined"]["cov_ln"])

args["Ki_gmean"] = Q_(data["kinetic_constants"]["KI"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KI"]["unit"])
args["Ki_ln_cov"] = np.array(data["kinetic_constants"]["KI"]["combined"]["cov_ln"])

args["conc_met_gmean"] = Q_(data["metabolite_concentrations"]["combined"]["geom_mean"], data["metabolite_concentrations"]["unit"])
args["conc_met_gstd"] = np.array(data["metabolite_concentrations"]["combined"]["geom_std"])

args["conc_enz_gmean"] = Q_(data["enzyme_concentrations"]["combined"]["geom_mean"], data["metabolite_concentrations"]["unit"])
args["conc_enz_gstd"] = np.array(data["enzyme_concentrations"]["combined"]["geom_std"])

args["rate_law"] = "CM"

metabolite_names = data["network"]["metabolite_names"]
reaction_names = data["network"]["reaction_names"]

In [5]:
mb = ModelBalancingConvex(**args)
mb.initialize_with_gmeans()
print("kcat reverse comparison:")
print("from JSON:  ", args["kcatr_gmean"].m_as("1/s"))
print("calculated: " , np.exp(mb.ln_kcatr.value))

/home/eladn/git/model-balancing/python/model_balancing/model_balancing_cvx.py:239: RuntimeWarning: invalid value encountered in log
  return np.log(self.fluxes.m_as("M/s")) - cp.vstack([ln_kcatf] * self.Ncond).T
/home/eladn/git/model-balancing/venv/lib/python3.9/site-packages/cvxpy/atoms/elementwise/log.py:34: RuntimeWarning: invalid value encountered in log
  return np.log(values[0])


ValueError: Parameter value must be real.

In [4]:
print("enzyme concentration comparison:")
print("from JSON:  \n", args["conc_enz_gmean"].m_as("M"))
print("calculated: \n" , np.exp(mb.ln_conc_enz.value))

enzyme concentration comparison:
from JSON:  
 [[1.74991311e-06 1.58257659e-06 5.01929073e-07 4.41456915e-07]
 [1.44387710e-06 3.31349350e-06 1.36297300e-06 1.43373983e-06]
 [9.39168869e-07 3.45175677e-07 1.29820842e-06 1.23089965e-06]]
calculated: 
 [[1.74820954e-06 1.58036686e-06 5.01694593e-07 4.41213742e-07]
 [1.44394132e-06 3.31662521e-06 1.36332578e-06 1.43442925e-06]
 [9.39100790e-07 3.45080990e-07 1.29855222e-06 1.23140778e-06]]


In [5]:
%%time

mb = ModelBalancingConvex(**args)

print("Setting enzyme conc. and kcatr gmeans to calculated values...")
# check that the driving forces at the mode are all positive
assert mb.is_gmean_feasible()

mb.initialize_with_gmeans()
print(np.exp(mb.ln_kcatr_gmean.value))

print(f"initial total squared Z-scores = {mb.objective_value:.3f}")
mb.print_z_scores()

Setting enzyme conc. and kcatr gmeans to calculated values...
[ 97.62328423   7.22549632 138.09477545]
initial total squared Z-scores = 0.000
enz = 0.0
met = 0.0
Keq = 0.0
kcatf = 0.0
kcatr = 0.0
Km = 0.0
Ka = 0.0
Ki = 0.0
CPU times: user 435 ms, sys: 7.82 ms, total: 443 ms
Wall time: 436 ms


In [6]:
mb.solve()
print(f"optimized total squared Z-scores = {mb.objective_value:.3f}")
print("-"*25 + " All Z-scores " + "-"*25)
mb.print_z_scores()

optimized total squared Z-scores = 0.000
------------------------- All Z-scores -------------------------
enz = 0.0
met = 0.0
Keq = 0.0
kcatf = 0.0
kcatr = 0.0
Km = 0.0
Ka = 0.0
Ki = 0.0


In [7]:
%%time

mb = ModelBalancingConvex(**args)
mb.solve()
print(f"optimized total squared Z-scores = {mb.objective_value:.3f}")
print("-"*25 + " All Z-scores " + "-"*25)
mb.print_z_scores()

optimized total squared Z-scores = 0.000
------------------------- All Z-scores -------------------------
enz = 0.0
met = 0.0
Keq = 0.0
kcatf = 0.0
kcatr = 0.0
Km = 0.0
Ka = 0.0
Ki = 0.0
CPU times: user 4.16 s, sys: 505 ms, total: 4.66 s
Wall time: 4.5 s


In [8]:
sbtabdoc = SBtab.SBtabDocument(name="balanced_states")

flux_df = pd.DataFrame(
    mb.fluxes.m_as("mM/s"),
    columns=["S1", "S2", "S3", "S4"]
)
flux_df.insert(0, "!QuantityType", "rate of reaction")
flux_df.insert(1, "!Reaction", ["E1", "E2", "E3"])
flux_sbtab = SBtab.SBtabTable.from_data_frame(
    flux_df.astype(str),
    table_id="MetabolicFlux",
    table_name="MetabolicFlux",
    table_type="QuantityMatrix",
    unit="mM/s",
)
sbtabdoc.add_sbtab(flux_sbtab)

c = Q_(np.exp(mb.ln_conc_met.value), "M")
conc_met_df = pd.DataFrame(
    c.m_as("mM"),
    columns=["S1", "S2", "S3", "S4"]
)
conc_met_df.insert(0, "!QuantityType", "concentration")
conc_met_df.insert(1, "!Compound", ["X1", "S2", "X3", "X4"])
conc_met_sbtab = SBtab.SBtabTable.from_data_frame(
    conc_met_df.astype(str),
    table_id="MetaboliteConcentrations",
    table_name="Metabolite concentrations",
    table_type="QuantityMatrix",
    unit="mM",
)
sbtabdoc.add_sbtab(conc_met_sbtab)

e = Q_(np.exp(mb.ln_conc_enz.value), "M")
conc_enz_df = pd.DataFrame(
    e.m_as("mM"),
    columns=["S1", "S2", "S3", "S4"]
)
conc_enz_df.insert(0, "!QuantityType", "concentration of enzyme")
conc_enz_df.insert(1, "!Reaction", ["E1", "E2", "E3"])
conc_enz_sbtab = SBtab.SBtabTable.from_data_frame(
    conc_enz_df.astype(str),
    table_id="EnzymeConcentrations",
    table_name="Enzyme concentrations",
    table_type="QuantityMatrix",
    unit="mM",
)
sbtabdoc.add_sbtab(conc_enz_sbtab)

gibbs_energy_df = pd.DataFrame(
    -mb.driving_forces.value,
    columns=["S1", "S2", "S3", "S4"]
)
gibbs_energy_df.insert(0, "!QuantityType", "Gibbs energy of reaction")
gibbs_energy_df.insert(1, "!Reaction", ["E1", "E2", "E3"])
gibbs_energy_sbtab = SBtab.SBtabTable.from_data_frame(
    gibbs_energy_df.astype(str),
    table_id="ReactionGibbsFreeEnergy",
    table_name="Gibbs free energies of reaction",
    table_type="QuantityMatrix",
    unit="kJ/mol",
)
sbtabdoc.add_sbtab(gibbs_energy_sbtab)

kcatf = Q_(np.exp(mb.ln_kcatf.value), "1/s")
kcatr = Q_(np.exp(mb.ln_kcatr.value), "1/s")
kcat_df = pd.DataFrame(
    np.vstack([kcatf.m_as("1/s"), kcatr.m_as("1/s")]).T,
    columns=["kcatf", "kcatr"]
)
kcat_df.insert(0, "!QuantityType", "catalytic rate constant")
kcat_df.insert(1, "!Reaction", ["E1", "E2", "E3"])
kcat_sbtab = SBtab.SBtabTable.from_data_frame(
    kcat_df.astype(str),
    table_id="CatalyticRateConstant",
    table_name="Catalytic rate constant",
    table_type="QuantityMatrix",
    unit="1/s",
)
sbtabdoc.add_sbtab(kcat_sbtab)

Km = Q_(np.exp(mb._create_dense_matrix(mb.S, mb.ln_Km).value), "M")
Km_df = pd.DataFrame(
    Km.m_as("mM").T,
    columns=["X1", "S2", "X3", "X4"]
)
Km_df.insert(0, "!QuantityType", "Michaelis constant")
Km_df.insert(1, "!Reaction", ["E1", "E2", "E3"])
Km_sbtab = SBtab.SBtabTable.from_data_frame(
    Km_df.astype(str),
    table_id="MichaelisConstant",
    table_name="Michaelis-Menten constant",
    table_type="QuantityMatrix",
    unit="mM",
)
sbtabdoc.add_sbtab(Km_sbtab)

Ka = Q_(np.exp(mb._create_dense_matrix(mb.S, mb.ln_Ka)), "M")
Ka_df = pd.DataFrame(
    Ka.m_as("mM").T,
    columns=["X1", "S2", "X3", "X4"]
)
Ka_df.insert(0, "!QuantityType", "activation constant")
Ka_df.insert(1, "!Reaction", ["E1", "E2", "E3"])
Ka_sbtab = SBtab.SBtabTable.from_data_frame(
    Ka_df.astype(str),
    table_id="ActivationConstant",
    table_name="Activation constant",
    table_type="QuantityMatrix",
    unit="mM",
)
sbtabdoc.add_sbtab(Ka_sbtab)

Ki = Q_(np.exp(mb._create_dense_matrix(mb.S, mb.ln_Ki)), "M")
Ki_df = pd.DataFrame(
    Ki.m_as("mM").T,
    columns=["X1", "S2", "X3", "X4"]
)
Ki_df.insert(0, "!QuantityType", "inhibition constant")
Ki_df.insert(1, "!Reaction", ["E1", "E2", "E3"])
Ki_sbtab = SBtab.SBtabTable.from_data_frame(
    Ki_df.astype(str),
    table_id="InhibitionConstant",
    table_name="Inhibition constant",
    table_type="QuantityMatrix",
    unit="mM",
)
sbtabdoc.add_sbtab(Ki_sbtab)

True

In [9]:
print(sbtabdoc.to_str())
sbtabdoc.write("balanced_states_cvx.tsv");

!!!SBtab DocumentName='balanced_states' SBtabVersion='1.0' Date='2021-4-5'
!!SBtab TableID='MetabolicFlux' TableType='QuantityMatrix' TableName='MetabolicFlux' SBtabVersion='1.0' Unit='mM/s'
!QuantityType	!Reaction	S1	S2	S3	S4
rate of reaction	E1	0.021387373764911267	0.01460085302465973	0.011737199044558173	0.009199216810779395
rate of reaction	E2	0.021387372521010195	0.014588598605683439	0.011737199044527745	0.009199199058691014
rate of reaction	E3	0.02138737477565732	0.014313621496418424	0.011737199044230595	0.009199219113649869

!!SBtab TableID='MetaboliteConcentrations' TableType='QuantityMatrix' TableName='Metabolite concentrations' SBtabVersion='1.0' Unit='mM'
!QuantityType	!Compound	S1	S2	S3	S4
concentration	X1	6.376118804041782	10.748467242280432	13.831139243968131	10.698178860894133
concentration	S2	0.15999513383275463	0.337406399956373	0.07259622428086908	0.10057633324432365
concentration	X3	1.1233084551350212	4.703127578088726	0.7065781144829528	1.166876240668952
concentrati